In [ ]:
import requests
from bs4 import BeautifulSoup
import requests
import re
import getopt,sys
import time
import pickle

header = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:20.0) Gecko/20100101 Firefox/20.0'}
url = 'http://www.sports-reference.com/cfb/schools/utah'
data= {}
data['years'] = {}
data['opponents'] = {}

start = time.time()
raw_html = requests.get(url, headers=header)
parsed = BeautifulSoup(raw_html.text, 'html.parser')
utah = parsed.find(id='div_utah')
utah_table = utah.find('tbody')
print('Getting years')
for yearRow in utah_table.find_all('tr', {'class': False}): 
    sys.stdout.write('.')
    sys.stdout.flush()
    yearCol = yearRow.find('td',{'data-stat': 'year_id'})
    year = yearCol.a.text
    data['years'][year] = {}
    data['years'][year]['conference'] = yearRow.find('td', {'data-stat': 'conf_id'}).a.text
    data['years'][year]['wins'] = yearRow.find('td', {'data-stat': 'wins'}).text
    data['years'][year]['losses'] = yearRow.find('td', {'data-stat': 'losses'}).text

print('\n')
print('Getting schedules')
for year, yearData in data['years'].items():
    sys.stdout.write('.')
    sys.stdout.flush()
#    print(year)
    yearUrl = url+'/'+year+'-schedule.html'
    yearRaw = requests.get(yearUrl, headers=header)
    yearParsed = BeautifulSoup(yearRaw.text, 'html.parser')
    data['years'][year]['schedule'] = []
    tb = yearParsed.find(id='div_schedule').find(id='schedule').thead.find_next_sibling()
    for gameData in tb.find_all('tr'):
        # Parse game
        game = {}
        
        # Get date
        gameDate = gameData.find('td', {'data-stat': 'date_game'}).text
        game['date'] = gameDate
        
        # Get opponent
        opponentRaw = gameData.find('td', {'data-stat': 'opp_name'}).text.replace('\xa0', ' ')
        opponent = re.sub(r'\(\d+\)', '',opponentRaw).strip()
        game['opponent'] = opponent
        
        # Get result
        gameResult = gameData.find('td', {'data-stat': 'game_result'}).text
        game['result'] = gameResult
        
        # Add to list
        data['years'][year]['schedule'].append(game)
        
        # Add to opponent table
        if not opponent in data['opponents']:
 #           print('Adding: '+opponent)
            data['opponents'][opponent] = []
            
        # Get points
        points = gameData.find('td', {'data-stat': 'points'}).text
        oppPoints = gameData.find('td', {'data-stat': 'opp_points'}).text
        oppGame = {}
        oppGame['date'] = gameDate
        oppGame['result'] = gameResult
        oppGame['points'] = points
        oppGame['opp_points'] = oppPoints
        data['opponents'][opponent].append(oppGame)
    
end = time.time()
print('')
print(end - start)


sys.setrecursionlimit(10000)
pickle.dump(data, open('utah_all.p', 'wb'))
print('done')
print(data['years']['2008'])
print(data['opponents']['Brigham Young'])


In [65]:
import sys
import pickle

alldata = pickle.load(open('utah_all.p', 'rb'))
print(alldata['2004'])


{'page': '225', 'games': [{'year': '2004', 'date': 'Sep 2', 'opponent': '21', 'result': 'W', 'utah': '41'}, {'year': '2004', 'date': 'Sep 11', 'opponent': '6', 'result': 'W', 'utah': '23'}, {'year': '2004', 'date': 'Sep 18', 'opponent': '6', 'result': 'W', 'utah': '48'}, {'year': '2004', 'date': 'Sep 25', 'opponent': '35', 'result': 'W', 'utah': '49'}, {'year': '2004', 'date': 'Oct 1', 'opponent': '7', 'result': 'W', 'utah': '28'}, {'year': '2004', 'date': 'Oct 16', 'opponent': '16', 'result': 'W', 'utah': '46'}, {'year': '2004', 'date': 'Oct 23', 'opponent': '28', 'result': 'W', 'utah': '63'}, {'year': '2004', 'date': 'Oct 30', 'opponent': '28', 'result': 'W', 'utah': '51'}, {'year': '2004', 'date': 'Nov 6', 'opponent': '31', 'result': 'W', 'utah': '63'}, {'year': '2004', 'date': 'Nov 13', 'opponent': '28', 'result': 'W', 'utah': '45'}, {'year': '2004', 'date': 'Nov 20', 'opponent': '21', 'result': 'W', 'utah': '52'}, {'year': '2004', 'date': 'Jan 1', 'opponent': '7', 'result': 'W', '

In [66]:
import requests
from bs4 import BeautifulSoup as bs4
import re
import pickle

# Chrome 60 user agent
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.78 Safari/537.36'}

data = {}
data['years'] = {}
data['errors'] = []

def displayError(errorString):
    print(errorString)
    data['errors'].append(errorString)

for sch in range(115,420): 
    sched = str(sch)
    url = 'https://www.utahutes.com/schedule.aspx?schedule='+sched
    page=requests.get(url, headers=header)
    if page.status_code == 200:
        parsed=bs4(page.text, 'html.parser')
        scheduleTable = parsed.find('article', {'class', 'sidearm-schedule'})
        scheduleType = scheduleTable.find('h2')
        if 'Football' in scheduleType.text:
            year = re.search('([0-9]+).*',scheduleType.text).group(1)
            data['years'][year] = {}
            data['years'][year]['page'] = sched
            data['years'][year]['games'] = []
            print(year)
            for game in scheduleTable.find_all('li', {'class': 'sidearm-schedule-game'}):
                gameRow = game.find('div', {'class': 'sidearm-schedule-game-row'})
                opponent = gameRow.find('div', {'class': 'sidearm-schedule-game-opponent'})
                details = opponent.find('div', {'class': 'sidearm-schedule-game-opponent-details'})
                gameDateHtml = details.find('div', {'class': 'sidearm-schedule-game-opponent-date'}).find('span')
                gameDate = re.search('([a-zA-Z]+ [0-9]+) .*', gameDateHtml.text.strip()).group(1)
                oppHtml = details.find('div', {'class': 'sidearm-schedule-game-opponent-text'})
                oppName = oppHtml.find('span', {'class': 'sidearm-schedule-game-opponent-name'})
                detailsHtml = gameRow.find('div',{'class': 'sidearm-schedule-game-details'})
                resultHtml = detailsHtml.find('div', {'class': 'sidearm-schedule-game-result'})
                if resultHtml:
                    if len(resultHtml.contents) == 9:
                        result = re.search('(.*),',resultHtml.contents[3].text).group(1)
                        capture = re.search('(.*)\-(.*)',resultHtml.contents[5].text)
                        utahScore = capture.group(1)
                        oppScore = capture.group(2)
                    else:
                        displayError(year+': Missing result for '+oppName.text.strip())
                        result = 'W'
                        utahScore = 'N/A'
                        oppScore = 'N/A'
                    game = {}
                    game['year'] = year
                    game['date'] = gameDate
                    game['opponent'] = oppName.text.strip()
                    game['result'] = result
                    game['utah'] = utahScore
                    game['opponent'] = oppScore
                    data['years'][year]['games'].append(game)
                else:
                    displayError(year+': Missing result HTML for '+oppName.text.strip())

pickle.dump(data,open('utah_all.p', 'wb'))
print(data['years']['2004'])
print(data['errors'])


1892
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
1999: Missing result for Louisiana-Monroe
2000
2001
2001: Missing result for Shrine Game
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2014: Missing result for Red/White Spring Game
2015
2015: Missing result for Utah Spring Football
2015: Missing result HTML for Brigham Young
2015: Missing result HTML for Spring Game
2016
{'page': '225', 'games': [{'year': '2004', 'date': 'Sep 2', 'opponent': '21', 'result': 'W', 'utah': '41'}, {'year': '2004', 'd